In [ ]:
import geopandas as gpd
import pandas as pd
import requests
import math
import numpy as np
from datetime import datetime
from shapely import wkt
from shapely.errors import WKTReadingError
from shapely.geometry import mapping, shape
from shapely import Polygon
import warnings
import os
import re
warnings.filterwarnings("ignore")

In [ ]:
login = {
    "username":"Dorito",
    "password":"Portador123"
}
# Crear una sesión para mantener la cookie
session = requests.Session()
response = session.post("https://192.168.179.3/api/user/login/", login, verify=False)

def get_data_paginated(url):
    login = {
        "username":"Dorito",
        "password":"Portador123"
    }
    # Crear una sesión para mantener la cookie
    session = requests.Session()
    response = session.post("https://192.168.179.3/api/user/login/", login, verify=False)
    datos = []
    while url:
        response = session.get(url, verify=False)
        info = response.json()
        datos.extend(info["results"])
        url = info["next"]
    return datos

def get_data_not_paginated(url):
    login = {
        "username":"Dorito",
        "password":"Portador123"
    }
    # Crear una sesión para mantener la cookie
    session = requests.Session()
    response = session.post("https://192.168.179.3/api/user/login/", login, verify=False)
    response = session.get(url, verify=False)
    info = response.json()
    return info


def parse_geom_safe(wkt_string):
    try:
        if isinstance(wkt_string, str) and "SRID=" in wkt_string:
            return wkt.loads(wkt_string.split(";", 1)[1])
    except:
        pass
    return None  # simplemente ignora lo inválido

<H3>SCRIPT PARA BUSCAR INFO</H3>

In [ ]:
# Obtener y normalizar datos
url = "https://192.168.179.3/api/v1/poligonos_lotes/"
url_productor = "https://192.168.179.3/api/v1/productor/"
url_extensionista = "https://192.168.179.3/api/v1/Extensionista/"
url_productor_data = "https://192.168.179.3/api/v1/productor/mongo_get/productor_data/"
url_fincas = "https://192.168.179.3/api/v1/poligonos_fincas/"
url_municipio = "https://192.168.179.3/api/v1/municipio/"
url_departamento = "https://192.168.179.3/api/v1/vereda/"
url_lotes = "https://192.168.179.3/api/v1/poligonos_lotes/mongo_get/mongo_atribute/"
print("cargando datos lotes")
datos = get_data_paginated(url)
print("cargando datos productor")
datos_productor = get_data_not_paginated(url_productor)
print("cargando datos extensionista")
datos_extensionista = get_data_not_paginated(url_extensionista)
print("cargando datos productor")
datos_productor_data = get_data_not_paginated(url_productor_data)
print("cargando datos fincas")
datos_fincas = get_data_paginated(url_fincas)
print("cargando datos lotes")
datos_lotes = get_data_not_paginated(url_lotes)
print("Cargando datos municipios")
datos_municipios = get_data_not_paginated(url_municipio)
print("cargando datos departamentos")
datos_departamentos = get_data_not_paginated(url_departamento)



df = pd.json_normalize(datos)
df_productor = pd.json_normalize(datos_productor)
df_extensionista = pd.json_normalize(datos_extensionista)
df_productor_data = pd.json_normalize(datos_productor_data)
df_fincas = pd.json_normalize(datos_fincas)
df_lotes = pd.json_normalize(datos_lotes)
df_municipio = pd.json_normalize(datos_municipios)
df_departamento = pd.json_normalize(datos_departamentos)


In [ ]:
df_municipio.to_excel("./municipios.xlsx")

In [ ]:
# Filtrar productores que tienen postgres_data.id
# df_productor = df_productor[df_productor["id"].notna()]
# Realizar los merge sin eliminar filas del DataFrame principal
df_completo = df.merge(
    df_productor,
    left_on="productor",
    right_on="id",
    how="left"
)

df_completo = df_completo.merge(
    df_productor_data,
    left_on="productor_data",
    right_on="_id",
    how="left"
)

df_completo = df_completo.merge(
    df_extensionista,
    left_on="extensionista",
    right_on="id",
    how="left"
)

# df_completo = df_completo.merge(
#     df_fincas,
#     left_on="finca",
#     right_on="id",
#     how="left"
# )

df_completo = df_completo.merge(
    df_lotes,
    left_on="id_x",
    right_on="poligono_id",
    how="left"
)

# Ejemplo: renombrar id en municipio y departamento
df_municipio = df_municipio.rename(columns={"id": "id_municipio"})
df_departamento = df_departamento.rename(columns={"id": "id_departamento"})
df_departamento = df_departamento.rename(columns={"municipio": "alkdsjflkajsñldkfjasdl"})


df_completo = df_completo.merge(
    df_municipio,
    left_on="municipio",
    right_on="id_municipio",
    how="left"
)

df_completo = df_completo.merge(
    df_departamento,
    left_on="vereda",
    right_on="id_departamento",
    how="left"
)

In [ ]:
def parse_geom_safe(wkt_string):
    try:
        if isinstance(wkt_string, str) and "SRID=" in wkt_string:
            return wkt.loads(wkt_string.split(";", 1)[1])
    except:
        pass
    return None  # simplemente ignora lo inválido

# Aplicar solo a las válidas
df_completo["geometry"] = df_completo["poligono"].apply(parse_geom_safe)

# Crear el GeoDataFrame sin que explote
gdf = gpd.GeoDataFrame(df_completo, geometry="geometry", crs="EPSG:4326")

In [ ]:
gdf.columns

In [ ]:
# Obtener y normalizar datos
url = "https://192.168.179.3/api/v1/poligonos_fincas/"
url_productor = "https://192.168.179.3/api/v1/productor/"
url_lotes = "https://192.168.179.3/api/v1/poligonos_fincas/mongo_get/mongo_atribute/"
print("cargando datos fincas")
datos = get_data_paginated(url)
print("cargando datos productor")
datos_productor = get_data_not_paginated(url_productor)
print("cargando datos fincas atributos")
datos_lotes = get_data_not_paginated(url_lotes)

df = pd.json_normalize(datos)
df_productor = pd.json_normalize(datos_productor)
df_lotes = pd.json_normalize(datos_lotes)

# Filtrar productores que tienen postgres_data.id
df_productor = df_productor[df_productor["id"].notna()]
# Realizar los merge sin eliminar filas del DataFrame principal
df_completo = df.merge(
    df_productor,
    left_on="productor",
    right_on="id",
    how="left"
)

df_completo = df_completo.merge(
    df_lotes,
    left_on="id_x",
    right_on="poligono_id",
    how="left"
)

df_completo = df_completo.merge(
    df_extensionista,
    left_on="extensionista",
    right_on="id",
    how="left"
)

# Aplicar solo a las válidas
df_completo["geometry"] = df_completo["poligono"].apply(parse_geom_safe)

# Crear el GeoDataFrame sin que explote
gdf_fincas = gpd.GeoDataFrame(df_completo, geometry="geometry", crs="EPSG:4326")

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from shapely.validation import make_valid

def filtrar_poligonos_superpuestos(gdf, umbral=0.01):
    gdf = gdf.copy()

    # Asegurar que la fecha sea datetime
    gdf["fecha"] = pd.to_datetime(
        gdf["fecha"], format="%d/%m/%Y %H:%M:%S", errors="coerce"
    )

    # Reparar geometrías inválidas
    gdf["geometry"] = gdf["geometry"].apply(
        lambda geom: make_valid(geom) if not geom.is_valid else geom
    )

    # Ordenar por fecha descendente (más recientes primero)
    gdf = gdf.sort_values(by="fecha", ascending=False).reset_index(drop=True)

    indices_validos = []

    for i, row_i in gdf.iterrows():
        print(f"{i}/{len(gdf)-1}")
        geom_i = row_i.geometry
        if geom_i is None or geom_i.is_empty:
            continue

        area_i = geom_i.area
        keep = True

        for j in indices_validos:
            geom_j = gdf.loc[j, "geometry"]
            if geom_j is None or geom_j.is_empty:
                continue

            area_j = geom_j.area

            try:
                inter = geom_i.intersection(geom_j).area
            except Exception as e:
                # Si no se puede intersectar, saltamos
                print(f"Error intersectando {i} y {j}: {e}")
                continue

            if inter > 0:
                overlap = inter / min(area_i, area_j)
                if overlap >= umbral:
                    keep = False
                    break

        if keep:
            indices_validos.append(i)

    return gdf.loc[indices_validos].reset_index(drop=True)


def filtrar_poligonos_priorizar_pequenos(gdf, umbral=0.01):
    gdf = gdf.copy()

    # Reparar geometrías
    gdf["geometry"] = gdf["geometry"].apply(lambda g: make_valid(g))

    # Calcular áreas
    gdf["area"] = gdf.geometry.area

    # Pequeños primero
    gdf = gdf.sort_values("area", ascending=True).reset_index(drop=True)

    indices_validos = []

    for i, row_i in gdf.iterrows():
        geom_i = row_i.geometry
        area_i = row_i.area

        keep = True
        
        # Comparar con los ya aceptados (que son pequeños o iguales)
        for j in indices_validos:
            geom_j = gdf.loc[j, "geometry"]
            area_j = gdf.loc[j, "area"]

            # No necesitamos verificar el caso contrario: j siempre es pequeño o igual
            inter = geom_i.intersection(geom_j).area
            if inter > 0:
                overlap = inter / min(area_i, area_j)
                if overlap >= umbral:
                    # geom_i (que es más grande o igual) se elimina
                    keep = False
                    break
        
        if keep:
            indices_validos.append(i)

    return gdf.loc[indices_validos].reset_index(drop=True)



In [ ]:
gdf_fincas = filtrar_poligonos_superpuestos(gdf_fincas,0.10)

In [ ]:
df_habilidad = pd.read_excel("~/Descargas/HABILIDAD NOVIEMBRE 2025 COOCENTRAL.xlsx", sheet_name="BASE")

In [ ]:
df_habilidad

In [ ]:
lista_cedulas = df_habilidad["CEDULA"].to_list()

In [ ]:
gdf_fincas["documento_x"]

In [ ]:
reporte = []
arreglo_productores_extensionista = []
gdf = gdf.loc[gdf["documento_x"].isin(lista_cedulas)]
gdf_fincas = gdf_fincas.loc[gdf_fincas["documento_x"].isin(lista_cedulas)]
df_productor = df_productor.loc[df_productor["documento"].isin(lista_cedulas)]
for i, row in df_extensionista.iterrows():
    gdf_lotes_extensionista = gdf.loc[gdf["extensionista"] == row["id"]]
    numero_productores_visitados = len(gdf_lotes_extensionista["productor"].unique())
    gdf_fincas_extensionista = gdf_fincas.loc[gdf_fincas["extensionista"] == row["id"]]
    productores_extensionista = df_productor.loc[df_productor["" \
    "extensionista"] == row["id"]]
    print(gdf_lotes_extensionista)
    reporte.append({
        "extensionista":row["first_name"] + " " +row["last_name"],
        "numero_productores_con_poligono": numero_productores_visitados,
        "numero_lotes_con_poligonos": len(gdf_lotes_extensionista),
        "area_lotes_con_poligonos": gdf_lotes_extensionista["area"].sum(),
        "numero_fincas_con_poligono": len(gdf_fincas_extensionista),
        "area_fincas_con_poligonos": gdf_fincas_extensionista["area"].sum(),
        "numero_productores_totales_BD_Sept": len(productores_extensionista),
        "porcentaje_visitados": (numero_productores_visitados/len(productores_extensionista))*100
    })

gdf_lotes_extensionista = gdf.loc[gdf["extensionista"].isna()]
numero_productores_visitados = len(gdf_lotes_extensionista["productor"].unique())
gdf_fincas_extensionista = gdf_fincas.loc[gdf_fincas["extensionista"].isna()]
productores_extensionista = df_productor.loc[df_productor["extensionista"].isna()]
# reporte.append({
#     "extensionista": "Desconocido",
#     "numero_productores_visitados": numero_productores_visitados,
#     "numero_productores_totales": len(productores_extensionista),
#     # "porcentaje_visitados": (numero_productores_visitados/len(productores_extensionista))*100,
#     "numero_lotes_visitados": len(gdf_lotes_extensionista),
#     "numero_fincas_visitadas": len(gdf_fincas_extensionista)
# })

gdf_lotes_extensionista = gdf
numero_productores_visitados = len(gdf_lotes_extensionista["productor"].unique())
gdf_fincas_extensionista = gdf_fincas
productores_extensionista = df_productor
reporte.append({
    "extensionista": "Totales",
    "numero_productores_con_poligono": numero_productores_visitados,
    "numero_lotes_con_poligonos": len(gdf_lotes_extensionista),
    "area_lotes_con_poligonos": gdf_lotes_extensionista["area"].sum(),
    "numero_fincas_con_poligono": len(gdf_fincas_extensionista),
    "area_fincas_con_poligonos": gdf_fincas_extensionista["area"].sum(),
    "numero_productores_totales_BD_Sept": len(productores_extensionista),
    "porcentaje_visitados": (numero_productores_visitados/len(productores_extensionista))*100
})

In [ ]:
df_reportes = pd.DataFrame(reporte)

In [ ]:
df_reportes.to_excel("./REPORTE_ING_YANETH_15_12_2025.xlsx")

In [ ]:
lkajsfdñlkañjsd

<h2>AQUI COMIENZA EL ANALISIS DE VEREDAS</h2>

In [ ]:
import geopandas as gpd
import pandas as pd
import requests
import math
import numpy
from datetime import datetime
from shapely import wkt
from shapely.errors import WKTReadingError
from shapely.geometry import mapping, shape
from shapely import Polygon
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import json

def parse_json_safe(x):
    if isinstance(x, dict):
        return x
    try:
        return json.loads(x)
    except (json.JSONDecodeError, TypeError):
        return {}

In [ ]:
login = {
    "username":"Dorito",
    "password":"Portador123"
}
# Crear una sesión para mantener la cookie
session = requests.Session()
response = session.post("https://produccion.local/api/user/login/", login, verify=False)
def get_data_paginated(url):
    login = {
        "username":"Dorito",
        "password":"Portador123"
    }
    # Crear una sesión para mantener la cookie
    session = requests.Session()
    response = session.post("https://produccion.local/api/user/login/", login, verify=False)
    datos = []
    while url:
        response = session.get(url, verify=False)
        info = response.json()
        datos.extend(info["results"])
        url = info["next"]
    return datos

def get_data_not_paginated(url):
    login = {
        "username":"Dorito",
        "password":"Portador123"
    }
    # Crear una sesión para mantener la cookie
    session = requests.Session()
    response = session.post("https://produccion.local/api/user/login/", login, verify=False)
    response = session.get(url, verify=False)
    info = response.json()
    return info

In [ ]:
# Obtener y normalizar datos
url = "https://produccion.local/api/v1/poligonos_lotes/"
url_productor = "https://produccion.local/api/v1/productor/"
url_productor_data = "https://produccion.local/api/v1/productor/mongo_get/productor_data/"
url_fincas = "https://produccion.local/api/v1/poligonos_fincas/"
url_municipio = "https://produccion.local/api/v1/municipio/"
url_departamento = "https://produccion.local/api/v1/vereda/"
url_lotes = "https://produccion.local/api/v1/poligonos_lotes/mongo_get/mongo_atribute/"
print("cargando datos lotes")
datos = get_data_paginated(url)
print("cargando datos productor")
datos_productor = get_data_not_paginated(url_productor)
print("cargando datos productor")
datos_productor_data = get_data_not_paginated(url_productor_data)
print("cargando datos fincas")
datos_fincas = get_data_paginated(url_fincas)
print("cargando datos lotes")
datos_lotes = get_data_not_paginated(url_lotes)
print("Cargando datos municipios")
datos_municipios = get_data_not_paginated(url_municipio)
print("cargando datos departamentos")
datos_departamentos = get_data_not_paginated(url_departamento)



df = pd.json_normalize(datos)
df_productor = pd.json_normalize(datos_productor)
df_productor_data = pd.json_normalize(datos_productor_data)
df_fincas = pd.json_normalize(datos_fincas)
df_lotes = pd.json_normalize(datos_lotes)
df_municipio = pd.json_normalize(datos_municipios)
df_departamento = pd.json_normalize(datos_departamentos)


In [ ]:
df_municipio.to_excel("./municipios.xlsx")

In [ ]:
df_departamento.to_excel("./veredas_base_datos.xlsx")

In [ ]:
# Filtrar productores que tienen postgres_data.id
df_productor = df_productor[df_productor["id"].notna()]
# Realizar los merge sin eliminar filas del DataFrame principal
df_completo = df.merge(
    df_productor,
    left_on="productor",
    right_on="id",
    how="left"
)

df_completo = df_completo.merge(
    df_productor_data,
    left_on="productor_data",
    right_on="_id",
    how="left"
)

df_completo = df_completo.merge(
    df_fincas,
    left_on="finca",
    right_on="id",
    how="left"
)

df_completo = df_completo.merge(
    df_lotes,
    left_on="id_x",
    right_on="poligono_id",
    how="left"
)

# Ejemplo: renombrar id en municipio y departamento
df_municipio = df_municipio.rename(columns={"id": "id_municipio"})
df_departamento = df_departamento.rename(columns={"id": "id_departamento"})
df_departamento = df_departamento.rename(columns={"municipio": "alkdsjflkajsñldkfjasdl"})


df_completo = df_completo.merge(
    df_municipio,
    left_on="municipio_x",
    right_on="id_municipio",
    how="left"
)

df_completo = df_completo.merge(
    df_departamento,
    left_on="vereda_x",
    right_on="id_departamento",
    how="left"
)



In [ ]:
# Construir el GeoDataFrame (con geometrías inválidas como None)

def parse_geom_safe(wkt_string):
    try:
        if isinstance(wkt_string, str) and "SRID=" in wkt_string:
            return wkt.loads(wkt_string.split(";", 1)[1])
    except:
        pass
    return None  # simplemente ignora lo inválido

# Aplicar solo a las válidas
df_completo["geometry"] = df_completo["poligono_x"].apply(parse_geom_safe)

# Crear el GeoDataFrame sin que explote
gdf = gpd.GeoDataFrame(df_completo, geometry="geometry", crs="EPSG:4326")

# Validación de cantidad
# assert len(gdf) == 10530, f"Se esperaban 10530 registros, pero hay {len(gdf)}"

In [ ]:
df_habilidad = pd.read_excel(r"C:\Users\dorito\Downloads\HABILIDAD SEPTIEMBRE 2025 COOCENTRAL(3.622) - UNIDAD TECNICA (1).xlsx", sheet_name="BASE ")

In [ ]:
# lista_ext_nuevos = ['YEISON FABIAN FIGUEROA SAENZ', 'JHOAN JERONIMO ORDOÑEZ MUÑOZ','ALEXANDER TORREJANO', 'CESAR POVEDA SILVA']
# df_nuevas_zonas = df_habilidad.loc[df_habilidad["TECNICO DE LA ZONA 2025"].isin(lista_ext_nuevos)]
df_nuevas_zonas = df_habilidad

In [ ]:
lista_cedulas = df_nuevas_zonas["CEDULA"].to_list()

In [ ]:
df_departamento.columns = ["id_vereda", "vereda_nombre", "poligono_vereda", "poligono_municipio"]

In [ ]:
gdf_filtro = gdf.loc[gdf["documento"].isin(lista_cedulas)]
gdf_filtro.head(5)

In [ ]:
df_habilidad.columns

In [ ]:
informacion_en_limpio = []
for i, row in df_nuevas_zonas.iterrows():
    gdf_filtro_productor = gdf_filtro.loc[gdf_filtro["documento"] == row["CEDULA"]]
    vereda = row['VEREDA ']
    municipio = row['MUNICIPIO']
    if not gdf_filtro_productor.empty:
        vereda = df_departamento["id_vereda"].loc[df_departamento["id_vereda"] == gdf_filtro_productor["vereda_x"].iloc[0]].iloc[0]
        municipio = df_departamento["poligono_municipio"].loc[df_departamento["id_vereda"] == gdf_filtro_productor["vereda_x"].iloc[0]].iloc[0]
    informacion_en_limpio.append({
        "documento": row["CEDULA"],
        "nombre_completo": row["NOMBRES Y APELLIDOS"],
        "municipio": municipio,
        "vereda": vereda,
        "extensionista": row['TECNICO DE LA ZONA 2025']
    })

In [ ]:
df_veredas_asociados = pd.DataFrame(informacion_en_limpio)

In [ ]:
df_departamento.columns = ['id_vereda', 'vereda', 'poligono_vereda', 'id_municipio']

In [ ]:
from shapely import wkt
import geopandas as gpd

# 1. Convertir WKT con SRID a geometrías Shapely
def limpiar_wkt(wkt_text):
    if isinstance(wkt_text, str):
        # Quitar el SRID=4326; si existe
        if wkt_text.upper().startswith("SRID"):
            wkt_text = wkt_text.split(";", 1)[1]
        return wkt.loads(wkt_text)
    return None

df_departamento["geometry"] = df_departamento["poligono_vereda"].apply(limpiar_wkt)
gdf_veredas = gpd.GeoDataFrame(df_departamento, geometry="geometry", crs=4326)
df_municipio["geometry"] = df_municipio["poligono_municipio"].apply(limpiar_wkt)
gdf_municipio = gpd.GeoDataFrame(df_municipio, geometry="geometry", crs=4326)

In [ ]:
from fuzzywuzzy import process

for i, row in df_veredas_asociados.iterrows():
    municipio = row["municipio"]

    # ------------------------------
    # 1️⃣ PROCESAR MUNICIPIO (igual que antes)
    # ------------------------------
    if isinstance(municipio, str):
        filtro = gdf_municipio[gdf_municipio["municipio"].str.lower() == municipio.lower()]

        if len(filtro) > 0:
            municipio = filtro.iloc[0]["id_municipio"]
            df_veredas_asociados.at[i, "municipio"] = municipio
        else:
            print(f"No se encontró municipio exacto: {municipio}")
            continue

    # ------------------------------
    # 2️⃣ PROCESAR VEREDA (exacto + fuzzy local)
    # ------------------------------
    vereda = row["vereda"]

    # Solo procesamos si es string
    if isinstance(vereda, str):

        # Veredas del municipio correcto
        df_vereda_filtro = gdf_veredas.loc[gdf_veredas["id_municipio"] == municipio]

        # 2A. MATCH EXACTO
        filtro = df_vereda_filtro[df_vereda_filtro["vereda"].str.lower() == vereda.lower()]

        if len(filtro) > 0:
            df_veredas_asociados.at[i, "vereda"] = filtro.iloc[0]["id_vereda"]
            continue

        # 2B. FUZZY MATCH SOLO DENTRO DEL MUNICIPIO
        lista_veredas = df_vereda_filtro["vereda"].tolist()

        if lista_veredas:
            mejor_match, score = process.extractOne(vereda, lista_veredas)

            if score >= 80:
                vereda_id = df_vereda_filtro.loc[
                    df_vereda_filtro["vereda"] == mejor_match,
                    "id_vereda"
                ].iloc[0]

                df_veredas_asociados.at[i, "vereda"] = vereda_id
                continue

        # ------------------------------------------------------------
        # 🚫 No match aceptable → dejamos el texto original
        # ------------------------------------------------------------
        print(f"No hay match válido para '{vereda}', se conserva el texto original.")
        # no se cambia nada
        continue


In [ ]:
print(df_veredas_asociados.columns)
print(df_municipio.columns)
print(df_departamento.columns)

In [ ]:
df_veredas_asociados["vereda"].unique()

In [ ]:
df_veredas_asociados_georeferenciadas = df_veredas_asociados.loc[~df_veredas_asociados["vereda"].apply(lambda x: isinstance(x, str))]

df_veredas_asociados_no_georeferenciadas = df_veredas_asociados.loc[df_veredas_asociados["vereda"].apply(lambda x: isinstance(x, str))]


In [ ]:
df_veredas_asociados_no_georeferenciadas

In [ ]:
# df_veredas_asociados_georeferenciadas.merge(df_departamento[["id_vereda", "vereda","poligono_vereda"]], left_on="vereda", right_on="id_vereda", how="left")
df_veredas_asociados_georeferenciadas = df_veredas_asociados_georeferenciadas.merge(df_departamento[["id_vereda", "vereda","poligono_vereda"]], left_on="vereda", right_on="id_vereda", how="left")
df_veredas_asociados_georeferenciadas = df_veredas_asociados_georeferenciadas.drop(columns=["id_vereda"])
df_veredas_asociados_georeferenciadas = df_veredas_asociados_georeferenciadas.rename({"vereda_x":"id_vereda", "vereda_y": "nombre_vereda"})

In [ ]:
df_veredas_asociados_georeferenciadas = df_veredas_asociados_georeferenciadas.rename(
    columns={
        "vereda_x": "id_vereda",
        "vereda_y": "nombre_vereda"
    }
)


In [ ]:
df_veredas_asociados_georeferenciadas

In [ ]:
df_veredas_asociados_georeferenciadas["poligono_vereda"] = df_veredas_asociados_georeferenciadas["poligono_vereda"].apply(limpiar_wkt)

# 2. Crear el GeoDataFrame
gdf_final = gpd.GeoDataFrame(df_veredas_asociados_georeferenciadas, geometry="poligono_vereda", crs="EPSG:4326")

gdf_final


In [ ]:
veredas_count = []
for vereda in gdf_final["id_vereda"].unique():
    gdf_filtro = gdf_final.loc[gdf_final["id_vereda"] == vereda]
    veredas_count.append({
        "id_vereda": vereda,
        "nombre_vereda": gdf_filtro["nombre_vereda"].iloc[0].upper(),
        "numero_asociados": len(gdf_filtro),
        "geometry":gdf_filtro["poligono_vereda"].iloc[0]
    })

gdf_final_veredas = gpd.GeoDataFrame(veredas_count, geometry="geometry", crs=4326)

In [ ]:
gdf_final_veredas.columns

In [ ]:
import geopandas as gpd
import pandas as pd

# Copia por seguridad
gdf_final = gdf_final_veredas.copy()

# Para ir guardando resultados temporales
rows_to_add = []

# Iterar solo sobre veredas con numero_asociados
for idx, row in gdf_final_veredas.iterrows():

    geom = row["geometry"]
    nombre = row["nombre_vereda"]

    # Buscar veredas vecinas
    vecinas = gdf_veredas[
        (gdf_veredas.geometry.touches(geom)) &
        (gdf_veredas["vereda"] != nombre)
    ]
    vecinas["VECINA"] = "SI"
    # Añadir cada vecina como fila nueva
    for _, r in vecinas.iterrows():
        rows_to_add.append(r)

# Convertir a GeoDataFrame
gdf_vecinas = gpd.GeoDataFrame(rows_to_add, geometry="geometry", crs=gdf_veredas.crs)
gdf_vecinas = gdf_vecinas.drop(columns=["poligono_vereda"])
gdf_vecinas = gdf_vecinas.rename(columns={
    "vereda": "nombre_vereda"
})

# Unir con gdf_final
gdf_final_extendido = pd.concat([gdf_final_veredas, gdf_vecinas], ignore_index=True)



In [ ]:
gdf_final_extendido

In [ ]:
gdf_final_extendido

In [ ]:
# gdf_final_extendido.columns = ["id_vereda", "vereda", "geometry", "id_municipio", "numero_asociados", "geometry", "VECINA"]

In [ ]:
# 1) Crear un filtro para las filas protegidas
protegidas = (
    (gdf_final_extendido["VECINA"] == "SI") |
    (gdf_final_extendido["numero_asociados"].notna() & (gdf_final_extendido["numero_asociados"] != 0))
)

# 2) Separar protegidas y no protegidas
gdf_protegidas = gdf_final_extendido[protegidas]
gdf_no_protegidas = gdf_final_extendido[~protegidas]

# 3) Hacer drop duplicates SOLO en no protegidas
gdf_no_protegidas = gdf_no_protegidas.drop_duplicates(subset="geometry")

# 4) Volver a unir
gdf_final_extendido = pd.concat([gdf_protegidas, gdf_no_protegidas], ignore_index=True)


In [ ]:
gdf_final_extendido

In [ ]:
for i, row in gdf_final_extendido.iterrows():
    if not pd.notna(row["id_municipio"]):
        df_filtro = df_departamento.loc[df_departamento["id_vereda"] == row["id_vereda"]]
        gdf_final_extendido.at[i,"id_municipio"]= df_filtro["id_municipio"].iloc[0]

In [ ]:
gdf_final_extendido

In [ ]:
gdf_final_extendido["municipio"] = None
gdf_final_extendido["departamento"] = None
for i, row in gdf_final_extendido.iterrows(): 
    id_municipio = row["id_municipio"]
    nombre = df_municipio["municipio"].loc[df_municipio["id_municipio"] == id_municipio].iloc[0]
    departamento = df_municipio["departamento"].loc[df_municipio["id_municipio"] == id_municipio].iloc[0]
    print(id_municipio)
    print(nombre)
    gdf_final_extendido.at[i, "municipio"] = nombre
    gdf_final_extendido.at[i, "departamento"] = departamento


In [ ]:
gdf_final_extendido

In [ ]:
gdf_filtro_asociados = gdf_final_extendido.loc[~gdf_final_extendido["numero_asociados"].isna()]
lista_veredas = gdf_filtro_asociados["id_vereda"].to_list()
gdf_filtro_vecinos = gdf_final_extendido.loc[gdf_final_extendido["numero_asociados"].isna()]
gdf_filtro_vecinos = gdf_filtro_vecinos.loc[~gdf_filtro_vecinos["id_vereda"].isin(lista_veredas)]
gdf_final_extendido = gpd.GeoDataFrame(pd.concat([gdf_filtro_asociados, gdf_filtro_vecinos]), geometry="geometry", crs=4326)

In [ ]:
gdf_final_extendido

In [ ]:
lista_municipios = gdf_final_extendido["id_municipio"].to_list()
lista_veredas = gdf_final_extendido["id_vereda"].to_list()

gdf_veredas_faltantes = gdf_veredas.loc[
    (~gdf_veredas["id_vereda"].isin(lista_veredas))&
    (gdf_veredas["id_municipio"].isin(lista_municipios))
]
gdf_veredas_faltantes = gdf_veredas_faltantes.rename(columns={"vereda":"nombre_vereda"})
gdf_final_extendido = gpd.GeoDataFrame(pd.concat([gdf_final_extendido, gdf_veredas_faltantes]), geometry="geometry", crs=4326)

In [ ]:
gdf_veredas

In [ ]:
gdf_final_extendido["municipio"] = None
gdf_final_extendido["departamento"] = None
for i, row in gdf_final_extendido.iterrows(): 
    id_municipio = row["id_municipio"]
    nombre = df_municipio["municipio"].loc[df_municipio["id_municipio"] == id_municipio].iloc[0]
    departamento = df_municipio["departamento"].loc[df_municipio["id_municipio"] == id_municipio].iloc[0]
    print(id_municipio)
    print(nombre)
    gdf_final_extendido.at[i, "municipio"] = nombre
    gdf_final_extendido.at[i, "departamento"] = departamento


In [ ]:
gdf_final_extendido.to_file("./veredas_influyentes_2.shp")

In [ ]:
with pd.ExcelWriter("Productores por vereda.xlsx", engine="openpyxl") as writer:
    df_veredas_asociados_georeferenciadas.to_excel(
        writer, 
        sheet_name="georeferenciadas", 
        index=False
    )

    df_veredas_asociados_no_georeferenciadas.to_excel(
        writer, 
        sheet_name="no_georeferenciadas", 
        index=False
    )
